In [1]:
import gns3fy
from gns3fy import Gns3Connector
from gns3fy import Node, Link

In [56]:
server = Gns3Connector('http://localhost:3080')
lab = gns3fy.Project(name='test2', connector=server)
lab.get()

In [58]:
pid = lab.project_id

In [4]:
lab.status

'opened'

In [15]:
lab.links_summary()

alpine1: Ethernet1 ---- host1: Ethernet0
alpine1: Ethernet2 ---- host2: Ethernet0
Switch4: Ethernet0 ---- PC3: Ethernet0
Switch4: Ethernet1 ---- PC8: Ethernet0
Switch1: Ethernet0 ---- PC2: Ethernet0
Switch1: Ethernet1 ---- PC7: Ethernet0
Switch2: Ethernet0 ---- PC1: Ethernet0
Switch2: Ethernet1 ---- PC6: Ethernet0
Switch3: Ethernet0 ---- PC4: Ethernet0
Switch3: Ethernet1 ---- PC5: Ethernet0
Switch4: Ethernet2 ---- Switch1: Ethernet2
Switch1: Ethernet3 ---- Switch2: Ethernet2
Switch2: Ethernet3 ---- Switch3: Ethernet2
Switch3: Ethernet3 ---- alpine1: Ethernet0


In [29]:
s2 = Node(project_id=pid, name='s2', connector=server, template='Ethernet switch')
s2.create()

In [65]:
host1 = Node(project_id=pid, name='host1', connector=server, template='VPCS')
host2 = Node(project_id=pid, name='host2', connector=server, template='VPCS')

In [7]:
# host1.create()
# host2.create()

host1.get()
host2.get()

In [8]:
host1.ports

[{'adapter_number': 0,
  'data_link_types': {'Ethernet': 'DLT_EN10MB'},
  'link_type': 'ethernet',
  'name': 'Ethernet0',
  'port_number': 0,
  'short_name': 'e0'}]

In [9]:
# nodes = [
#     dict(node_id=alpine1.node_id, adapter_number=0, port_number=1),
#     dict(node_id=host1.node_id, adapter_number=0, port_number=0)
# ]

# h1_ap1_link = Link(project_id=pid, nodes=nodes, connector=server)
# h1_ap1_link.create()

In [10]:
# nodes = [
#     dict(node_id=alpine1.node_id, adapter_number=0, port_number=2),
#     dict(node_id=host2.node_id, adapter_number=0, port_number=0)
# ]

# h2_ap1_link = Link(project_id=pid, nodes=nodes, connector=server)
# h2_ap1_link.create()

In [10]:
import telnetlib
import time

In [86]:
h1_console = telnetlib.Telnet('127.0.0.1', 5002, timeout=30)
h1_console.write(b'show ip\n')
data = h1_console.read_until(b'EOF\n', timeout=0.1)
f = open('ips', 'w')
f.write(data.decode('utf-8'))
f.close()
h1_console.close()

In [106]:
import re
pattern = r'(\d+.\d+.\d+.\d+/\d+)'
match = re.search(pattern, data.decode("utf-8"))
if match:
        print(match.group(1))

192.168.1.2/8


In [13]:
import re
def extract_ip(data):        
    pattern = r'(\d+.\d+.\d+.\d+/\d+)'
    match = re.search(pattern, data.decode("utf-8"))
    if match:
        return match.group(1)
    raise KeyError()

In [1]:
def get_ip(node) -> None:
    console = telnetlib.Telnet('127.0.0.1', node.console, timeout=5)
    console.write(b'show ip\n')
    data = console.read_until(b'EOF\n', timeout=0.1)
    try:
        ip = extract_ip(data)
    except KeyError:
        print(f'ip not found for node {node.name} with port: {node.console}')

    f = open('ips', 'a')
    f.write(f'{node.name}, {node.console}, {ip}\n')
    f.close()
    console.close()

def set_ip(node, ip_addr, gateway_addr) -> None:
    console = telnetlib.Telnet('127.0.0.1', node.console, timeout=5)
    cmd = f'ip {ip_addr} {gateway_addr}\n'
    console.write(cmd.encode('utf-8'))
    data = console.read_until(b'EOF\n', timeout=0.1)
    console.close()



In [7]:
pc4 = Node(project_id=pid, name='PC4', connector=server, template='VPCS')
pc4.get()

In [14]:
set_ip(pc4, '192.168.1.4', '255.0.0.0')

b'ip 192.168.1.4 255.0.0.0\r\nChecking for duplicate address...\r\n'


In [12]:
get_ip(pc4)

NameError: name 'extract_ip' is not defined

In [108]:
nodes = lab.nodes
for node in nodes:
    if node.node_type == 'vpcs':
        get_ip(node)

##### aight now to build the tree topology : )
##### soooo how would it look like sef

In [8]:
pc9 = Node(project_id=pid, name='pc9', connector=server, template='VPCS').create()


[]

In [10]:
lab.get()
lab.nodes

[Node(name='pc9', project_id='fdd91169-ceac-4c27-aacb-b3ad16ff725a', node_id='eddf4e60-5538-4032-8715-7831d3766cb9', compute_id='local', node_type='vpcs', node_directory='/home/g1f7/GNS3/projects/fdd91169-ceac-4c27-aacb-b3ad16ff725a/project-files/vpcs/eddf4e60-5538-4032-8715-7831d3766cb9', status='stopped', ports=[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet0', 'port_number': 0, 'short_name': 'e0'}], port_name_format='Ethernet{0}', port_segment_size=0, first_port_name=None, locked=False, label={'rotation': 0, 'style': None, 'text': 'pc9', 'x': None, 'y': -40}, console=5028, console_host='0.0.0.0', console_type='telnet', console_auto_start=False, command_line='', custom_adapters=[], height=59, width=65, symbol=':/symbols/vpcs_guest.svg', x=-687, y=-257, z=1, template_id='19021f99-e36f-394d-b4a1-8aaa902ab9cc', properties={}, template=None),
 Node(name='pc1', project_id='fdd91169-ceac-4c27-aacb-b3ad16ff725a', node_id='3238

In [59]:
#create vpcs(es)
for i in range(6):
    Node(project_id=pid, name=f'pc{i}', connector=server, template='VPCS').create()

#create switches
for i in range(2):
    Node(project_id=pid, name=f's{i}', connector=server, template='Ethernet switch').create()



In [94]:
lab.get()
lab.nodes

[Node(name='pc0', project_id='1d8eef6f-440f-4fdc-821a-acd539c6ce58', node_id='f921dc69-7c23-436d-a69f-1966a448ce06', compute_id='local', node_type='vpcs', node_directory='/home/g1f7/GNS3/projects/1d8eef6f-440f-4fdc-821a-acd539c6ce58/project-files/vpcs/f921dc69-7c23-436d-a69f-1966a448ce06', status='stopped', ports=[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet0', 'port_number': 0, 'short_name': 'e0'}], port_name_format='Ethernet{0}', port_segment_size=0, first_port_name=None, locked=False, label={'rotation': 0, 'style': 'font-family: TypeWriter;font-size: 10.0;font-weight: bold;fill: #000000;fill-opacity: 1.0;', 'text': 'pc0', 'x': 21, 'y': -25}, console=5028, console_host='0.0.0.0', console_type='telnet', console_auto_start=False, command_line='', custom_adapters=[], height=59, width=65, symbol=':/symbols/vpcs_guest.svg', x=0, y=0, z=1, template_id='19021f99-e36f-394d-b4a1-8aaa902ab9cc', properties={}, template=None),
 N

In [50]:
3//3

0

In [95]:
s = lab.nodes[6:]
n = lab.nodes[:6]





In [91]:
for i in range(3):
    Node(project_id=pid, name=f's{i}', connector=server, template='Ethernet switch').create()


In [96]:
s

[Node(name='s0', project_id='1d8eef6f-440f-4fdc-821a-acd539c6ce58', node_id='62231514-7213-4e83-b7dc-42113817b781', compute_id='local', node_type='ethernet_switch', node_directory=None, status='started', ports=[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet0', 'port_number': 0, 'short_name': 'e0'}, {'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet1', 'port_number': 1, 'short_name': 'e1'}, {'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet2', 'port_number': 2, 'short_name': 'e2'}, {'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet3', 'port_number': 3, 'short_name': 'e3'}, {'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet4', 'port_number': 4, 'short_name': 'e4'}, {'adapter_number': 

In [83]:
for i in range(len(n)):
    print(i%len(s) + 1)

1
2
1
2
1
2


In [86]:
for i in range(len(n)):
    port = (i%3) + 1 #3 here shows the number of hosts that would be connected to a switch.
    nodes = [
             dict(node_id=s[i//3].node_id, adapter_number=0, port_number=port),
             dict(node_id=n[i].node_id, adapter_number=0, port_number=0)
            ]
    
    Link(project_id=pid, nodes=nodes, connector=server).create()


In [98]:
for i in range(len(s)-1):
    port = (i % 2) + 4
    nodes = [
        dict(node_id=s[i].node_id, adapter_number=0, port_number=port),
        dict(node_id=s[i+1].node_id, adapter_number=0, port_number=port),        
    ]
    Link(project_id=pid, nodes=nodes, connector=server).create()
